# 使用GRU_CRF训练数据

In [3]:
# 使用Google的colab调用文件时需要添加路径，将文件放入指定的位置才能识别到
# from google.colab import drive
# drive.mount('/content/drive') 

# import sys
# sys.path.append('/content/drive/MyDrive/data/')

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
from gru_crf import BiGRUCRF
# 使用GPU
gpus = tf.config.experimental.list_physical_devices(device_type="GPU")
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type="GPU")

D:\Anaconda3\envs\resume8\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# 数据准备

In [2]:
# 加载数据
X = []
feaX = []
y = []

def data_read(line:str):
    lx, lfea, ly = eval(line)
    X.append(lx)
    feaX.append(lfea)
    y.append(ly)

In [3]:
%%time
# 读取数据并将提取出数据中的原始文本、文本特征及其对应的标签
# readline方法
# with open("label_old_only_work_project.txt", encoding='utf-8') as f:
#     line = f.readline()
#     while line:
#         data_read(line)
#         line = f.readline()
        
# readlines方法
with open("label_old_only_work_project.txt", encoding='utf-8') as f:
    for line in f.readlines():
        data_read(line)

# X[0], feaX[0], y[0]
len(feaX), len(y)
# X[:3]

CPU times: total: 3.67 s
Wall time: 3.7 s


(905, 905)

In [4]:
# # 数据提取
# X = [[
#         X[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#     ] for i in range(len(y)) if 'project-B' in y[i] or 'project-I' in y[i] or 'work-B' in y[i] or 'work-I' in y[i]
# ]

# feaX = [[
#         feaX[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#     ] for i in range(len(y)) if 'project-B' in y[i] or 'project-I' in y[i] or 'work-B' in y[i] or 'work-I' in y[i]
# ]

# y = [[
#         y[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#     ] for i in range(len(y)) if 'project-B' in y[i] or 'project-I' in y[i] or 'work-B' in y[i] or 'work-I' in y[i]
# ]

# # 验证数据提取正确
# X_len = np.array([len(i) for i in X])
# feaX_len = np.array([len(i) for i in feaX])
# y_len = np.array([len(i) for i in y])
# print('验证数据提取正确：', max(X_len - feaX_len)==0, max(feaX_len - y_len)==0, max(X_len - y_len)==0)
# print('提取后数据数量：', len(feaX), len(y))

In [5]:
%%time
#X = [[
#        X[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#    ] for i in range(len(y)) if 'project-B' in y[i] or 'work-B' in y[i]
#]
#
#feaX = [[
#        feaX[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#    ] for i in range(len(y)) if 'project-B' in y[i] or 'work-B' in y[i]
#]
#
#y = [[
#        y[i][j] for j in range(len(y[i])) if y[i][j] != 'O'
#    ] for i in range(len(y)) if 'project-B' in y[i] or 'work-B' in y[i] 
#]

X_new = []
feaX_new = []
y_new = []
for i in range(len(y)):
    if 'project-B' in y[i] or 'work-B' in y[i]:
        X_new.append([X[i][j] for j in range(len(y[i])) if y[i][j] != 'O'])
        feaX_new.append([feaX[i][j] for j in range(len(y[i])) if y[i][j] != 'O'])
        y_new.append([y[i][j] for j in range(len(y[i])) if y[i][j] != 'O'])

CPU times: total: 62.5 ms
Wall time: 52 ms


In [6]:
type(feaX_new)

list

In [7]:
# # get target encode dictionary
# tag2id = set()
# for i in y:
#     tag2id.update(i)
# tag2id = list(tag2id)
# tag2id.sort()
# tag2id = {k:v for v, k in enumerate(tag2id)}
# # 添加特征字符标签
# # tag2id['<start>'] = len(tag2id)
# tag2id['<PAD>'] = len(tag2id)

# # label encoding
# y_new = [[tag2id[token] for token in sequence] for sequence in y]

# tag2id #, y[0]

In [8]:
# 构建数据标签字典
tag2id = set()
for i in y_new:
    tag2id.update(i)
tag2id = list(tag2id)
tag2id.sort()
tag2id = {k:v for v, k in enumerate(tag2id)}
# 添加特征字符标签
# tag2id['<start>'] = len(tag2id)
tag2id['<PAD>'] = len(tag2id)

# label encoding
y_new = [[tag2id[token] for token in sequence] for sequence in y_new]

tag2id #, y[0]

{'project-B': 0, 'project-I': 1, 'work-B': 2, 'work-I': 3, '<PAD>': 4}

In [9]:
# # seqence padding
# max_len = max([len(i) for i in y])
# fea_dim = len(feaX[0][0])

# feaX = np.array(
#     [i + [[0] * fea_dim] * (max_len - len(i)) for i in feaX]
# )
# y = np.array(
#     [i + [tag2id['<PAD>']] * (max_len - len(i)) for i in y]
# )

# feaX.shape, y.shape

In [10]:
# seqence padding
max_len = max([len(i) for i in y_new])
fea_dim = len(feaX_new[0][0])

feaX_new = np.array(
    [i + [[0] * fea_dim] * (max_len - len(i)) for i in feaX_new]
)
y_new = np.array(
    [i + [tag2id['<PAD>']] * (max_len - len(i)) for i in y_new]
)

feaX_new.shape, y_new.shape

((891, 612, 24), (891, 612))

In [11]:
from gru_crf import *
model = BiGRUCRF(len(tag2id) - 1)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.AUC(),
                       tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])
model.fit(feaX_new, 
          y_new,
          batch_size = 40,
          epochs = 300,
          validation_split = 0.2,
          validation_freq = 20
         )
model.summary()

Epoch 1/300


OperatorNotAllowedInGraphError: in user code:

    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    D:\common_tools\common_tools\nlp\model\gru_crf_model\gru_crf.py:34 call  *
        X, mask = inputs
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:520 __iter__
        self._disallow_iteration()
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:513 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:489 _disallow_when_autograph_enabled
        raise errors.OperatorNotAllowedInGraphError(

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate

In [11]:
# # 数据划分
# feaX_train, feaX_valid, y_train, y_valid = train_test_split(
#     feaX, y, test_size=0.2, random_state=42
# )

# feaX_train.shape, y_train.shape, feaX_valid.shape, y_valid.shape

((712, 612, 24), (712, 612), (179, 612, 24), (179, 612))

In [22]:
# 数据划分
feaX_train, feaX_valid, y_train, y_valid = train_test_split(feaX_new, 
                                                            y_new, 
                                                            test_size=0.2, 
                                                            random_state=39)

feaX_train.shape, y_train.shape, feaX_valid.shape, y_valid.shape

((712, 612, 24), (712, 612), (179, 612, 24), (179, 612))

In [54]:
# 数据集生成
def data_generater(X, y, batch_size, tag2id, is_mask=True):
    sample_num = X.shape[0]
    data = []
    if is_mask:
        for i in range(int(sample_num/batch_size) + 1):
            if i * batch_size < sample_num:
                data.append([
                    tf.constant(X[i * batch_size : (i + 1) * batch_size])
                    , tf.constant(y[i * batch_size : (i + 1) * batch_size])
                    , tf.constant(np.where(y[i * batch_size : (i + 1) * batch_size] == tag2id['<PAD>'], 0., 1.))
                ])
    else:
        for i in range(int(sample_num/batch_size) + 1):
            if i * batch_size < sample_num:
                data.append([
                    tf.constant(X[i * batch_size : (i + 1) * batch_size])
                    , tf.constant(y[i * batch_size : (i + 1) * batch_size])
                ])
    return data


batch_size = 40
train_data = data_generater(feaX_train, y_train, batch_size, tag2id)
valid_data = [
    feaX_valid
    , y_valid
    , tf.constant(np.where(y_valid == tag2id['<PAD>'], 0., 1.))
]

# len(train_data), train_data[0], train_data[-1]
np.shape(train_data)

D:\Anaconda3\envs\resume8\lib\site-packages\numpy\core\fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(18, 3)

In [ ]:
from gru_crf import *
model = BiGRUCRF(len(tag2id) - 1)
# model.load_weights('./checkpoints/best_model')
model.train(train_data, valid_data, epochs=1000, verbose=0)

step: 100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:57<00:00,  3.19s/it]


epoch  0   tra_loss: 62.274944   val_loss: 53.83756   val_accuracy: 0.64296913   val_precision: 0.17974022   val_recall: 0.2507615   val_F1score: 0.2093925566363784 



step: 100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:54<00:00,  3.05s/it]


epoch  1   tra_loss: 51.34507   val_loss: 44.641464   val_accuracy: 0.7042248   val_precision: 0.391568   val_recall: 0.37051392   val_F1score: 0.38075012917492235 



step: 100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:54<00:00,  3.04s/it]


...

step: 100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:55<00:00,  3.11s/it]


epoch  18   tra_loss: 11.051776   val_loss: 9.048557   val_accuracy: 0.8671298   val_precision: 0.9352381   val_recall: 0.9182775   val_F1score: 0.9266802132996589 



step: 100%|████████████████████████████████████████████████████████████████████████████| 18/18 [00:55<00:00,  3.10s/it]


epoch  19   tra_loss: 10.408105   val_loss: 9.017732   val_accuracy: 0.87131995   val_precision: 0.9345613   val_recall: 0.92800134   val_F1score: 0.9312697760583517 



step:  94%|███████████████████████████████████████████████████████████████████████▊    | 17/18 [00:51<00:03,  3.02s/it]

In [50]:
# feaX_new = np.array([i for i in feaX_new])
# y_new = np.array([i for i in y_new])


Epoch 1/300


OperatorNotAllowedInGraphError: in user code:

    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    D:\common_tools\common_tools\nlp\model\gru_crf_model\gru_crf.py:34 call  *
        X, mask = inputs
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:520 __iter__
        self._disallow_iteration()
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:513 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    C:\Users\86183\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\ops.py:489 _disallow_when_autograph_enabled
        raise errors.OperatorNotAllowedInGraphError(

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate